In [1]:
##Checking my IP.Not necessary!

In [1]:
import socket

In [3]:
socket.gethostname() 

'DESKTOP-6TVJE19'

In [4]:
print(socket.gethostbyname(socket.gethostname()))

10.232.3.125


In [5]:
def get_host_ip():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(('8.8.8.8', 80))
        ip = s.getsockname()[0]
    finally:
        s.close()
    return ip

In [6]:
print(get_host_ip())

10.232.3.125


In [7]:
addrs = socket.getaddrinfo(socket.gethostname(),None)
for item in addrs:
    print(item)

(<AddressFamily.AF_INET6: 23>, 0, 0, '', ('fe80::38e5:c500:b2e4:55cc', 0, 0, 7))
(<AddressFamily.AF_INET: 2>, 0, 0, '', ('10.232.3.125', 0))


In [2]:
import urllib ##Basic use 
from bs4 import BeautifulSoup ##polishing html codes
from urllib import request
from urllib import parse
from urllib.request import urlopen
import re
import time
import datetime
import csv
import sys
import codecs
import string
import random
import socket
import http.cookiejar

In [14]:
##The IP agent which is obtained from agent website. Besides, we can purchase from T-Mall.
proxy_list={
    'http':'157.230.220.233:8080',   
    'http':'45.55.46.222:8080',  
    'https':'204.48.18.225:8080',
    'http':'198.211.109.90:8080',
    'https':'159.203.184.52:3128',
    'http':'157.230.210.133:8080',
    'http':'198.211.103.89:80',
    'http':'134.209.125.125:8080',
    'http':'157.230.232.130:8080',
    'http':'206.189.231.239:8080',
    'http':'104.248.7.88:3128'
            }

In [19]:
##Here is the headers we can use later.
##('User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36')

In [20]:
##example of how we use headers, to mimic human searching avoid being detected as machine.
url100='https://www.thestreet.com/quote/SCTY.html'
req100=urllib.request.Request(url100)
req100.add_header('User-Agent','Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36')
html100=urllib.request.urlopen(req100,timeout=500).read()
html100 = bytes.decode(html100,encoding="utf-8")
print(html100)
##without the headers we cannot get into TheStreet website. However,for Sec and Nasdaq dont worry about that.

<!DOCTYPE html><html lang="en">


<!--[if IE 8 ]>    <html lang="en" class="no-js no-css ie8" xmlns:fb="http://ogp.me/ns/fb#"> <![endif]-->
<!--[if IE 9 ]>    <html lang="en" class="no-js no-css ie9"    > <![endif]-->
<!--[if (gt IE 9)|!(IE)]><!--> <html lang="en" class="no-js no-css"><!--<![endif]-->
<head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# website: http://ogp.me/ns/website#">
    <script type="text/javascript">
        var dataLayer = [];
    </script>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/>
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=1024">
    <meta name="referrer" content="unsafe-url">
    <title>SolarCity Corp - NASDAQ:SCTY - Stock Quote & News - TheStreet</title>
    <meta property="fb:app_id" content="458919507482096"/>
    <meta property="fb:pages" content="112877175855"/>
    <link rel="dns-prefetch" href="//static.chartbeat.com"/><link rel="dns-prefetch" href="//pixel.

In [21]:
##Part I:
##Search from 2010 to 2018 from Nasdaq, and get all the IPO companies` information including the symbol, CIK, company name, business address, Tel-num, issue price, outstanding shares etc.

In [10]:
##creating an url list, which can help us to open hundreds of web pages to search for IPO companies from 2010 to 2018. 
y=['2010','2011','2012','2013','2014','2015','2016','2017','2018']
m=['01','02','03','04','05','06','07','08','09','10','11','12']
##url_list=prior url +year+month.
url_list=['https://www.nasdaq.com/markets/ipos/activity.aspx?tab=pricings&month='+a+'-'+b for a in y for b in m]
for item in url_list:
    print(item)

https://www.nasdaq.com/markets/ipos/activity.aspx?tab=pricings&month=2010-01
https://www.nasdaq.com/markets/ipos/activity.aspx?tab=pricings&month=2010-02
https://www.nasdaq.com/markets/ipos/activity.aspx?tab=pricings&month=2010-03
https://www.nasdaq.com/markets/ipos/activity.aspx?tab=pricings&month=2010-04
https://www.nasdaq.com/markets/ipos/activity.aspx?tab=pricings&month=2010-05
https://www.nasdaq.com/markets/ipos/activity.aspx?tab=pricings&month=2010-06
https://www.nasdaq.com/markets/ipos/activity.aspx?tab=pricings&month=2010-07
https://www.nasdaq.com/markets/ipos/activity.aspx?tab=pricings&month=2010-08
https://www.nasdaq.com/markets/ipos/activity.aspx?tab=pricings&month=2010-09
https://www.nasdaq.com/markets/ipos/activity.aspx?tab=pricings&month=2010-10
https://www.nasdaq.com/markets/ipos/activity.aspx?tab=pricings&month=2010-11
https://www.nasdaq.com/markets/ipos/activity.aspx?tab=pricings&month=2010-12
https://www.nasdaq.com/markets/ipos/activity.aspx?tab=pricings&month=2011-01

In [23]:
target_list=list()
for item0 in url_list:
    html0=urllib.request.urlopen(item0).read()
    soup0 = BeautifulSoup(html0)
    soup0.prettify()    
    for anchor in soup0.find_all('a', href=True): ##find the link to open new page from the previous one.
        a=anchor['href']
        if('https://www.nasdaq.com/markets/ipos/company/' in a):
            target=a
            target_list.append(target)

In [25]:
##create necessary information list we need, otherwise the loop only rememebr the last call.
symbol_list=list()
name_list=list()
address_list=list()
phone_list=list()
employees_list=list()
issueprice_list=list()
shareoutstanding_list=list()
CIK_list=list()
years_list=list()
for item1 in target_list:
    response1=urllib.request.urlopen(item1)
    soup1=BeautifulSoup(response1, 'html.parser')
    for title1 in soup1.find_all('title'):
        title1_txt=title1.get_text()
        symbol=re.findall(r'[(](.*?)[)]', title1_txt) ##pulling out the value here between "()" to get symbol.
        for table1 in soup1.find_all('table'):
                table1_txt=table1.get_text()
                if('Company Name' in table1_txt):
                        td1=table1.find_all('td')
                        for i in range(len(td1)):
                            tdtext1=td1[i].get_text()
                            if('Company Name' in tdtext1):
                                name=td1[i+1].get_text()
                                address=td1[i+3].get_text()
                                phone=td1[i+5].get_text()
                                employees=td1[i+11].get_text()
                                issueprice=td1[i+23].get_text()
                                shareoutstanding=td1[i+35].get_text()
                                CIK=td1[i+43].get_text()
                                years=td1[i+17].get_text()
                                name_list.append(name)
                                address_list.append(address)
                                phone_list.append(phone)
                                employees_list.append(employees)
                                issueprice_list.append(issueprice)
                                shareoutstanding_list.append(shareoutstanding)
                                CIK_list.append(CIK)
                                years_list.append(years)
                                symbol_list.append(symbol)

In [ ]:
##see the results and check correctness.
print(name_list)
print(address_list)
print(phone_list)
print(employees_list)
print(issueprice_list)
print(shareoutstanding_list)
print(CIK_list)
print(years_list)
print(symbol_list)

In [ ]:
##write a csv.file to record the results.
with open('information-final.csv','w') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(name_list)
    f_csv.writerow(address_list)
    f_csv.writerow(phone_list)
    f_csv.writerow(employees_list)
    f_csv.writerow(issueprice_list)
    f_csv.writerow(shareoutstanding_list)
    f_csv.writerow(CIK_list)
    f_csv.writerow(years_list)
    f_csv.writerow(symbol_list)

In [21]:
##Part II:
##Select the industry types we need:Technology, Telecom, Media and Biotech.

In [11]:
symbol_list=list()
for item2 in url_list:
    req = urllib.request.urlopen(item2).read()
    soup2 = BeautifulSoup(req)
    soup2.prettify()    
    for anchor2 in soup2.find_all('a', href=True):
        a=anchor2['href']
        if('https://www.nasdaq.com/symbol/' in a):
            smb=a
            symbol_list.append(smb)

In [12]:
Technology_list=list()
for s in symbol_list:
    page2 = urllib.request.urlopen(s).read()
    soup3 = BeautifulSoup(page2)
    soup3.prettify()
    for an in soup3.find_all('a', href=True):
        h=an['href']
        if('https://www.nasdaq.com/screening/companies-by-industry.aspx?industry=Technology'in h):
            select1=s
            Technology_list.append(select1)

URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>

In [ ]:
##check the tech list.
print(Technology_list)

In [ ]:
Healthcare_list=list()
for v in symbol_list:
    page3 = urllib.request.urlopen(v).read()
    soup4 = BeautifulSoup(page3)
    soup4.prettify()
    for anc in soup4.find_all('a', href=True):
        g=anc['href']
        if('https://www.nasdaq.com/screening/companies-by-industry.aspx?industry=Health%2bCare'in g):
            select2=v
            Healthcare_list.append(select2)

In [ ]:
##check the health list
print(Healthcare_list)

In [ ]:
fin1_list=list()
for t in range(len(Technology_list)):
    res1=Technology_list[t].split('/') ##get the symbol
    fin1=res1[len(res1)-1]
    fin1_list.append(fin1)

In [ ]:
print(fin1_list)

In [ ]:
fin2_list=list()
for j in range(len(Healthcare_list)):
    res2=Healthcare_list[j].split('/')
    fin2=res2[len(res2)-1]
    fin2_list.append(fin2)

In [ ]:
print(fin2_list)

In [ ]:
with open('Technology.csv','w') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(fin1_list)
    f_csv.writerow(Technology_list)

In [ ]:
with open('Healthcare.csv','w') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(fin2_list)
    f_csv.writerow(Healthcare_list)

In [23]:
##Part III:
##Get stock option price from S-1 form in SEC, and hen determine the lowball priced stock option group.

In [24]:
url_list=list()
with open('CIKlist.csv', newline='', encoding='UTF-8') as csvfile:
    reader=csv.reader(csvfile)
    for p in reader:
        u=str(p) ##we need to makee those string
        prunedd=u.strip(string.punctuation) ##sttrip all the unnecessary pounctuation.like ''.
        url='https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=000'+prunedd+'&type=S-1&dateb=&owner=exclude&count=40'
        url_list.append(url)

In [25]:
print(url_list)

['https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=000ufeffCIK&type=S-1&dateb=&owner=exclude&count=40', 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001421526&type=S-1&dateb=&owner=exclude&count=40', 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001477324&type=S-1&dateb=&owner=exclude&count=40', 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001477156&type=S-1&dateb=&owner=exclude&count=40', 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001473078&type=S-1&dateb=&owner=exclude&count=40', 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001295976&type=S-1&dateb=&owner=exclude&count=40', 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001403385&type=S-1&dateb=&owner=exclude&count=40', 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001477049&type=S-1&dateb=&owner=exclude&count=40', 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=000142

In [ ]:
page_list=list()
for item in url_list:
    response5=urllib.request.urlopen(item)
    soup = BeautifulSoup(response5,features="lxml")
    td=soup.find_all('td')       
    for i in range(len(td)):
        tdtxt=td[i].get_text()
        if('S-1'==tdtxt):
            tdn=td[i+1]
            for ah in tdn.find_all('a'):
                page=ah['href']
                page_list.append(page)

In [ ]:
print(page_list)

In [ ]:
url_list2=list()
for m in page_list:
    pa='https://www.sec.gov'+m
    url_list2.append(pa)

In [ ]:
print(url_list2)

In [ ]:
page1_list=list()
for h in url_list2:
    response6=urllib.request.urlopen(h)
    soup1 = BeautifulSoup(response6,features="lxml")
    td1=soup1.find_all('td')       
    for g in range(len(td1)):
        tdtxt1=td1[g].get_text()
        if('1'==tdtxt1):
            tdn1=td1[g+2]
            for ah1 in tdn1.find_all('a'):
                page1=ah1['href']
                page1_list.append(page1)

In [ ]:
print(page1_list)

In [ ]:
url_list3=list()
for n in page1_list:
    paa='https://www.sec.gov'+n
    url_list3.append(paa)

In [ ]:
print(url_list3)

In [ ]:
with open('urlsecondlevel.csv','w') as wow: ##We need to store the temporate results, because the next step the machine is gonna visit thousands of time.Not necessary if you have 4 hours seating infront of computer.
    wow_csv = csv.writer(wow)
    wow_csv.writerow(url_list3)

In [ ]:
##The urlsecondlevel.csv carries the same data with Level.csv.
dic={}
with open('Level.csv', newline='', encoding='UTF-8') as woww:
    readerw=csv.DictReader(woww)
    for row in readerw:
        dic[row['\ufeffCIK']]=row['URLF']

In [ ]:
for k,v in dic.items(): ##applying for dict to match each CIK and URL, because it contains dupicated url previously.
    print(k)
    print(v)

In [ ]:
stockoption_list=list()
CIKlist=list()
for k,v in dic.items():
    response7=urllib.request.urlopen(v)
    soup7 = BeautifulSoup(response7,features="lxml")
    td7=soup7.find_all('td')       
    for l in range(len(td7)):
        tdtxt7=td7[l].get_text()
        if('weighted average exercise price' in tdtxt7):##by searching weighted average excerse price we can have the stock option, incentive plan information with outstanding or granted prices.
            try:
                tdn7=td7[l].get_text()
            except IndexError:
                pass
            stockoption_list.append(tdn7)
            CIKlist.append(k)

In [ ]:
with open('NEWstockoption.csv', 'w', encoding='utf-8') as file:
    stock=csv.writer(file)
    stock.writerow(stockoption_list)
    stock.writerow(CIKlist)

In [27]:
##Part IV:
##Fetching PE ration, Forward PE ration, EPS from Nasdaq, Yahoo and The street.

In [ ]:
dic={}
with open('selected.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        dic[row['ï»¿Name']]=row['symbolurl']
        print(row)

In [ ]:
PEr_list=list()
CIKlist=list()
for k,v in dic.items(): 
    with urlopen(v) as text:
        soup = BeautifulSoup(text, 'html.parser')
        for table in soup.find_all('table'):
            table_txt=table.get_text()
            if('Forward P/E (1y)' in table_txt): ##fetching Forward PE ration from Nasdaq.
                td=table.find_all('td')
                for i in range(len(td)):
                    td_txt=td[i].get_text()
                    if('Forward P/E (1y)' in td_txt):
                        spanr=td[i].get_text()
                        PEr_list.append(spanr)
                        CIKlist.append(k)

In [ ]:
print(PEr_list)

In [ ]:
with open('PEPPratio.csv','w') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(PEr_list)
    f_csv.writerow(CIKlist)

In [ ]:
PEE_list=list()
CIK_list=list()
for k,v in dic.items(): 
    with urlopen(v) as text:
        soup = BeautifulSoup(text, 'html.parser')
        for table in soup.find_all('table'):
            table_txt=table.get_text()
            if('P/E Ratio' in table_txt): ##fetching real PE ration from Nasdaq.
                td=table.find_all('td')
                for i in range(len(td)):
                    td_txt=td[i].get_text()
                    if('P/E Ratio' in td_txt):
                        span=td[i].get_text()
                        PEE_list.append(span)
                        CIK_list.append(k)

In [ ]:
print(PEE_list)

In [ ]:
with open('PEEratio.csv','w') as ff:
    ff_csv = csv.writer(ff)
    ff_csv.writerow(PEE_list)
    ff_csv.writerow(CIK_list)

In [28]:
##some of the results from Nasdaq shows that some companies they have NE for both PE and Forward PE, so we decide to search the missing ones from Yahoo.

In [ ]:
dic={}
with open('notincluded.csv', newline='', encoding='UTF-8') as toy: ##not included in Previous results but still be our targets.
    readertoy=csv.DictReader(toy)
    for row in readertoy:
        dic[row['\ufeffSymbol']]=row['CIK']

In [ ]:
k_list=list()
for k,v in dic.items():
    url3='https://finance.yahoo.com/quote/'+k+'?p='+k+'&.tsrc=fin-srch'
    k_list.append(url3)

In [ ]:
print(k_list)

In [ ]:
CIK_list2=list()
EPSlist2=list()
name_list=list()
for k,v in dic.items():
    url3='https://finance.yahoo.com/quote/'+k+'?p='+k+'&.tsrc=fin-srch'
    with urlopen(url3) as text2:
        soup2 = BeautifulSoup(text2, 'html.parser')
        td1=soup2.find_all('td')
        for x in range(len(td1)):
            tdtxt=td1[x].get_text()
            if('EPS (TTM)' in tdtxt):
                eps=td1[x+1].get_text()
                CIK_list2.append(v)
                name_list.append(k)
                EPSlist2.append(eps)  

In [ ]:
with open('yahooeps.csv','w') as yahoo:
    yahoo_csv = csv.writer(yahoo)
    yahoo_csv.writerow(name_list)
    yahoo_csv.writerow(CIK_list2)
    yahoo_csv.writerow(EPSlist2)

In [32]:
##Part V:
##Get current stock prices, it could be updated timely.

In [3]:
def get_price(url1):
    with urlopen(url1) as text:
        soup = BeautifulSoup(text, 'html.parser')
        for p in soup.find_all('p'):
            p_txt=p.get_text()
            if('The current last sale of' in p_txt):
                spantxt=p.find_all('span')
                for i in range(len(spantxt)):
                    data=spantxt[i].get_text()
                    if('The current last sale of' in data):
                        try:
                            price=spantxt[i+1].get_text()
                            if("$" in price):
                                pruned_txt=price.strip(string.punctuation)
                                return pruned_txt # get rid of () $ punctuations
                        except IndexError:
                             pass

In [4]:
dic={}
with open('Currentprice-Selectedgroup.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        dic[row['symbolname']]=row['symbolurl']

In [14]:
price_list=list()
CIK_list=list()
Time_list=list()
for k,v in dic.items():
    url1="https://www.nasdaq.com/symbol/"+k
    p=get_price(url1)
    T=time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) ##get local time
    if(p is not None):
            rep=p.split('$')
            rep2=rep[len(rep)-1]
            rep3=float(rep2)
            price_list.append(rep3)
    CIK_list.append(k)
    Time_list.append(T)
    print("CIK:", k)
    print("Localtime:", T)
    print("Current Price:", rep3)

CIK: ALIM
Localtime: 2019-06-01 15:33:51
Current Price: 
                                    $0.88
                                
CIK: ACRX
Localtime: 2019-06-01 15:33:51
Current Price: 
                                    $2.59
                                
CIK: AL
Localtime: 2019-06-01 15:33:51
Current Price: 
                                    $36
                                
CIK: AMBA
Localtime: 2019-06-01 15:33:52
Current Price: 
                                    $37.91
                                
CIK: AGIO
Localtime: 2019-06-01 15:33:52
Current Price: 
                                    $46.17
                                
CIK: AAOI
Localtime: 2019-06-01 15:33:52
Current Price: 
                                    $8.66
                                
CIK: AERI
Localtime: 2019-06-01 15:33:53
Current Price: 
                                    $36.42
                                
CIK: AKER
Localtime: 2019-06-01 15:33:53
Current Price: 
                    

CIK: BITA
Localtime: 2019-06-01 15:34:13
Current Price: 
                                    $10.74
                                
CIK: IPHI
Localtime: 2019-06-01 15:34:13
Current Price: 
                                    $43.88
                                
CIK: LEDS
Localtime: 2019-06-01 15:34:13
Current Price: 
                                    $2.85
                                
CIK: INXN
Localtime: 2019-06-01 15:34:14
Current Price: 
                                    $73.71
                                
CIK: KIPS
Localtime: 2019-06-01 15:34:14
Current Price: None
CIK: PCRX
Localtime: 2019-06-01 15:34:14
Current Price: 
                                    $43.51
                                
CIK: TRNX
Localtime: 2019-06-01 15:34:15
Current Price: 
                                    $0.44
                                
CIK: NPTN
Localtime: 2019-06-01 15:34:15
Current Price: 
                                    $3.89
                                
CIK: CSOD
L

CIK: MODN
Localtime: 2019-06-01 15:34:37
Current Price: 
                                    $18.02
                                
CIK: TTPH
Localtime: 2019-06-01 15:34:37
Current Price: 
                                    $0.8091
                                
CIK: IPDN
Localtime: 2019-06-01 15:34:37
Current Price: 
                                    $2.84
                                
CIK: EVTC
Localtime: 2019-06-01 15:34:38
Current Price: 
                                    $28.66
                                
CIK: CMRX
Localtime: 2019-06-01 15:34:38
Current Price: 
                                    $3.40
                                
CIK: CGIX
Localtime: 2019-06-01 15:34:38
Current Price: 
                                    $0.1626
                                
CIK: EPZM
Localtime: 2019-06-01 15:34:39
Current Price: 
                                    $13.74
                                
CIK: KMDA
Localtime: 2019-06-01 15:34:39
Current Price: 
            

CIK: RVNC
Localtime: 2019-06-01 15:35:00
Current Price: 
                                    $10.77
                                
CIK: QURE
Localtime: 2019-06-01 15:35:00
Current Price: 
                                    $59.31
                                
CIK: GNCA
Localtime: 2019-06-01 15:35:00
Current Price: 
                                    $5.61
                                
CIK: BIOC
Localtime: 2019-06-01 15:35:01
Current Price: 
                                    $1.22
                                
CIK: HIVE
Localtime: 2019-06-01 15:35:01
Current Price: 
                                    $3.40
                                
CIK: TWOU
Localtime: 2019-06-01 15:35:01
Current Price: 
                                    $37.99
                                
CIK: WATT
Localtime: 2019-06-01 15:35:02
Current Price: 
                                    $4.12
                                
CIK: ATEN
Localtime: 2019-06-01 15:35:02
Current Price: 
                

CIK: CALA
Localtime: 2019-06-01 15:35:21
Current Price: 
                                    $5.26
                                
CIK: VBLT
Localtime: 2019-06-01 15:35:22
Current Price: 
                                    $1.37
                                
CIK: NEOT
Localtime: 2019-06-01 15:35:22
Current Price: None
CIK: EYES
Localtime: 2019-06-01 15:35:22
Current Price: 
                                    $0.7448
                                
CIK: FGEN
Localtime: 2019-06-01 15:35:22
Current Price: 
                                    $36.24
                                
CIK: PRAH
Localtime: 2019-06-01 15:35:23
Current Price: 
                                    $86.73
                                
CIK: UPLD
Localtime: 2019-06-01 15:35:23
Current Price: 
                                    $46.79
                                
CIK: CHRS
Localtime: 2019-06-01 15:35:23
Current Price: 
                                    $18.90
                                
CIK: NVRO

CIK: GBT
Localtime: 2019-06-01 15:35:43
Current Price: 
                                    $60.78
                                
CIK: ZYNE
Localtime: 2019-06-01 15:35:43
Current Price: 
                                    $12.36
                                
CIK: NTEC
Localtime: 2019-06-01 15:35:43
Current Price: 
                                    $5.16
                                
CIK: NBRV
Localtime: 2019-06-01 15:35:44
Current Price: 
                                    $2.29
                                
CIK: PEN
Localtime: 2019-06-01 15:35:44
Current Price: 
                                    $142.70
                                
CIK: RGNX
Localtime: 2019-06-01 15:35:44
Current Price: 
                                    $43.02
                                
CIK: MYOK
Localtime: 2019-06-01 15:35:45
Current Price: 
                                    $46.58
                                
CIK: IOTS
Localtime: 2019-06-01 15:35:45
Current Price: 
               

CIK: TTD
Localtime: 2019-06-01 15:36:04
Current Price: 
                                    $198.81
                                
CIK: EVBG
Localtime: 2019-06-01 15:36:04
Current Price: 
                                    $78.64
                                
CIK: BL
Localtime: 2019-06-01 15:36:04
Current Price: 
                                    $51.41
                                
CIK: QTNA
Localtime: 2019-06-01 15:36:05
Current Price: 
                                    $24.26
                                
CIK: MYOV
Localtime: 2019-06-01 15:36:05
Current Price: 
                                    $9.83
                                
CIK: RARX
Localtime: 2019-06-01 15:36:05
Current Price: 
                                    $21.63
                                
CIK: IRTC
Localtime: 2019-06-01 15:36:06
Current Price: 
                                    $68.46
                                
CIK: CRSP
Localtime: 2019-06-01 15:36:06
Current Price: 
               

CIK: FSCT
Localtime: 2019-06-01 15:36:24
Current Price: 
                                    $32.12
                                
CIK: EYE
Localtime: 2019-06-01 15:36:24
Current Price: 
                                    $27.23
                                
CIK: RMBL
Localtime: 2019-06-01 15:36:24
Current Price: 
                                    $4.41
                                
CIK: MDB
Localtime: 2019-06-01 15:36:25
Current Price: 
                                    $140.34
                                
CIK: OPTN
Localtime: 2019-06-01 15:36:25
Current Price: 
                                    $7.59
                                
CIK: KIDS
Localtime: 2019-06-01 15:36:26
Current Price: 
                                    $38.85
                                
CIK: HAIR
Localtime: 2019-06-01 15:36:26
Current Price: 
                                    $0.4639
                                
CIK: CARG
Localtime: 2019-06-01 15:36:26
Current Price: 
              

CIK: ASLN
Localtime: 2019-06-01 15:36:44
Current Price: 
                                    $2.91
                                
CIK: CBLK
Localtime: 2019-06-01 15:36:45
Current Price: 
                                    $15.05
                                
CIK: INSP
Localtime: 2019-06-01 15:36:45
Current Price: 
                                    $56.47
                                
CIK: UBX
Localtime: 2019-06-01 15:36:45
Current Price: 
                                    $8.16
                                
CIK: DOMO
Localtime: 2019-06-01 15:36:45
Current Price: 
                                    $33.50
                                
CIK: ENTX
Localtime: 2019-06-01 15:36:46
Current Price: 
                                    $3.45
                                
CIK: FTSV
Localtime: 2019-06-01 15:36:46
Current Price: 
                                    $11.31
                                
CIK: STIM
Localtime: 2019-06-01 15:36:46
Current Price: 
                

CIK: PLAN
Localtime: 2019-06-01 15:37:01
Current Price: 
                                    $43.51
                                
CIK: EQ
Localtime: 2019-06-01 15:37:02
Current Price: 
                                    $6.45
                                
CIK: ESTC
Localtime: 2019-06-01 15:37:02
Current Price: 
                                    $82.05
                                
CIK: GH
Localtime: 2019-06-01 15:37:02
Current Price: 
                                    $76.88
                                
CIK: KOD
Localtime: 2019-06-01 15:37:03
Current Price: 
                                    $9.88
                                
CIK: UPWK
Localtime: 2019-06-01 15:37:03
Current Price: 
                                    $14.98
                                
CIK: TLC
Localtime: 2019-06-01 15:37:03
Current Price: 
                                    $5.40
                                
CIK: TLSA
Localtime: 2019-06-01 15:37:04
Current Price: 
                     

In [13]:
with open('currentprice20190602.csv','w') as f:
    f_csv = csv.writer(f)
    for i in range(len(price_list)):
        f_csv.writerow([CIK_list[i], Time_list[i], price_list[i]])

AttributeError: 'NoneType' object has no attribute 'strip'